<a href="https://colab.research.google.com/github/ckdlsqpf/projectmanager/blob/main/KLUE_BERT_%E1%84%90%E1%85%A9%E1%84%8F%E1%85%B3%E1%86%AB_%E1%84%87%E1%85%AE%E1%86%AB%E1%84%85%E1%85%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate evaluate

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [ ]:
from transformers import BertForTokenClassification
model = BertForTokenClassification.from_pretrained(
    "klue/bert-base", num_labels=13, id2label={
    0: 'B-DT', 1: 'I-DT',  # 날짜(Date)
    2: 'B-LC',  3: 'I-LC', # 장소(Location)
    4: 'B-OG', 5: 'I-OG',  # 단체(Organization)
    6: 'B-PS',  7: 'I-PS', # 사람(Person)
    8: 'B-QT', 9: 'I-QT',  # 수량(Quantity)
    10: 'B-TI', 11: 'I-TI', # 시간(Time)
    12: 'O'})               # 기타(Outside)
model.to('cuda:0')

In [ ]:
from datasets import load_dataset
ner_train = load_dataset('klue', 'ner', split='train')
ner_val = load_dataset('klue', 'ner', split='validation')

In [5]:
ner_train[0]

{'sentence': '특히 <영동고속도로:LC> <강릉:LC> 방향 <문막휴게소:LC>에서 <만종분기점:LC>까지 <5㎞:QT> 구간에는 승용차 전용 임시 갓길차로제를 운영하기로 했다.',
 'tokens': ['특',
  '히',
  ' ',
  '영',
  '동',
  '고',
  '속',
  '도',
  '로',
  ' ',
  '강',
  '릉',
  ' ',
  '방',
  '향',
  ' ',
  '문',
  '막',
  '휴',
  '게',
  '소',
  '에',
  '서',
  ' ',
  '만',
  '종',
  '분',
  '기',
  '점',
  '까',
  '지',
  ' ',
  '5',
  '㎞',
  ' ',
  '구',
  '간',
  '에',
  '는',
  ' ',
  '승',
  '용',
  '차',
  ' ',
  '전',
  '용',
  ' ',
  '임',
  '시',
  ' ',
  '갓',
  '길',
  '차',
  '로',
  '제',
  '를',
  ' ',
  '운',
  '영',
  '하',
  '기',
  '로',
  ' ',
  '했',
  '다',
  '.'],
 'ner_tags': [12,
  12,
  12,
  2,
  3,
  3,
  3,
  3,
  3,
  12,
  2,
  3,
  12,
  12,
  12,
  12,
  2,
  3,
  3,
  3,
  3,
  12,
  12,
  12,
  2,
  3,
  3,
  3,
  3,
  12,
  12,
  12,
  8,
  9,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12]}

In [6]:
!git clone https://gist.github.com/220bdfde44d6fcdd78375877370a39f6.git preproc
!mv preproc/ner_preproc.py .

Cloning into 'preproc'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 12 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (12/12), done.
Resolving deltas: 100% (3/3), done.


In [7]:
from tokenizers.pre_tokenizers import BertPreTokenizer
import ner_preproc

preproc = ner_preproc.TokenAlignPreprocessor(
    tokenizer=tokenizer,
    pre_tokenizer=BertPreTokenizer(),
    outside_label_id=12)  # [TODO] O 레이블의 번호

train_ds = ner_train.map(preproc.convert_example)
eval_ds = ner_val.map(preproc.convert_example)

Map:   0%|          | 0/21008 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [8]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="test_trainer",
    num_train_epochs=1,
    evaluation_strategy="epoch")

from transformers import DataCollatorForTokenClassification
data_collator = \
    DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.055700,0.065942


TrainOutput(global_step=2626, training_loss=0.0776635062485729, metrics={'train_runtime': 349.8095, 'train_samples_per_second': 60.056, 'train_steps_per_second': 7.507, 'total_flos': 572113064143440.0, 'train_loss': 0.0776635062485729, 'epoch': 1.0})

In [9]:
from transformers import pipeline
ner = pipeline(
    'token-classification',
    model=model,
    tokenizer=tokenizer,
    device='cuda:0')

In [12]:
ner('''강남신세계백화점''')

[{'entity': 'B-LC',
  'score': 0.8572999,
  'index': 1,
  'word': '강남',
  'start': 0,
  'end': 2},
 {'entity': 'I-LC',
  'score': 0.8803305,
  'index': 2,
  'word': '##신',
  'start': 2,
  'end': 3},
 {'entity': 'I-LC',
  'score': 0.8424919,
  'index': 3,
  'word': '##세계',
  'start': 3,
  'end': 5},
 {'entity': 'I-LC',
  'score': 0.84702015,
  'index': 4,
  'word': '##백화점',
  'start': 5,
  'end': 8}]